## Google Colab configs

In [ ]:
# !git clone https://github.com/ReverendBayes95/Amazon_Reviews.git
# %cd Amazon_Reviews

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#path = "/content/drive/MyDrive/amazon_reviews_dataset/"

## Import libraries

In [1]:
import utils.data_io as data_io
import utils.filters as filters
import utils.aggregations as aggregations
import utils.models as models   
import variables
import torch 

c:\Users\aksha\Repositories\Amazon_Reviews\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read the reviews data

In [3]:
colnames = ['polarity', 'title', 'text']
train_df = data_io.read_reviews("C:/Users/aksha/Repositories/Amazon_Reviews/data/input/train.csv", colnames)
test_df = data_io.read_reviews("C:/Users/aksha/Repositories/Amazon_Reviews/data/input/test.csv", colnames)

In [4]:
train = aggregations.stratified_sample(train_df, 'polarity', 100000)
test = aggregations.stratified_sample(test_df, 'polarity', 100000)

del train_df, test_df

## Pre-process the reviews

In [5]:
train_pre_process = filters.TextPreProcess(train, variables.appos)
test_pre_process = filters.TextPreProcess(test, variables.appos)
train = train_pre_process.apply_text_filtering()
test = test_pre_process.apply_text_filtering()

train = aggregations.create_single_review(train)
test = aggregations.create_single_review(test)

## Predict the sentiment

### With transformers pipelines module

In [7]:
# from transformers import pipeline
# classifier = pipeline("text-classification")
# pipeline_preds = classifier(test_df['review'].tolist())

pipeline_preds = models.hf_classification_pipeline(test)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


KeyboardInterrupt: 